# Linear Regression

Housing Prices Competition for Kaggle Learn Users:
* https://www.kaggle.com/c/home-data-for-ml-course/data

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

from yellowbrick.target import FeatureCorrelation
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
house_data = pd.read_csv('home-train.csv')

# I'm going to ignore the test dataset so that we can manually go through the splitting process
# But later you could try using both the train and test set if you want
# house_data_test = pd.read_csv('home-test.csv')

## Major part of any data science / ML project:  Working with data

In [ ]:
house_data.head()

In [ ]:
house_data.info()

If we had more time, we could explore how to handle null values, alter datatypes as needed, come up with meaningful combinations of features, deal with value encodings, etc.

In [ ]:
h2 = house_data[['OverallQual','OverallCond','BedroomAbvGr','GrLivArea','SalePrice']].copy()

In [ ]:
h2.sample(10)

In [ ]:
sns.regplot(data=h2, x='OverallQual', y='SalePrice')

In [ ]:
sns.regplot(data=h2, x='OverallCond', y='SalePrice')

In [ ]:
sns.regplot(data=h2, x='BedroomAbvGr', y='SalePrice')

In [ ]:
sns.regplot(data=h2, x='GrLivArea', y='SalePrice')

In [ ]:
sns.boxplot(data=h2['GrLivArea'])

### Using one feature for simple linear regression

In [ ]:
X = h2[['GrLivArea']]
y = h2['SalePrice']

In [ ]:
X.head()

In [ ]:
y.head()

Remember to split the dataset into one set for training & validation and another set for testing.  This allows you to assess the generalization error of a trained model on data it has not "seen" yet.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# initialize the object for your model
linear_regression = LinearRegression()

# train the model -> this determines the optimal values of model coefficients
model = linear_regression.fit(X_train, y_train)

# assess the performance of your model
print("Testing score : ", linear_regression.score(X_test, y_test))

# use the trained model to make predictions
y_pred = model.predict(X_test)

# you can also use these predictions to assess performance
print("Testing score R2 : ", r2_score(y_test, y_pred))

That's all for the training.  We can follow that up with any number of subsequent assessments of what the trained model can do, what its predictions look like, etc.

In [ ]:
df = pd.DataFrame({'test': y_test, 'predicted': y_pred})

In [ ]:
df.sample(10)

In [ ]:
plt.scatter(X_test, y_test)
plt.plot(X_test, y_pred, c='r')

In [ ]:
print("Training score : ", linear_regression.score(X_train, y_train))
print("Testing score : ", linear_regression.score(X_test, y_test))

In [ ]:
r2score = r2_score(y_test, y_pred)
msescore = mean_squared_error(y_test, y_pred)

In [ ]:
print("Testing score R2 : ", r2score)
print("Testing score StdDev : ", np.sqrt(msescore))

In [ ]:
theta_0 = linear_regression.coef_
theta_0

In [ ]:
intercept = linear_regression.intercept_
intercept

In [ ]:
plt.plot(y_pred, label="Prediction")
plt.plot(y_test.values, label="Actual")
plt.legend()

In [ ]:
plt.plot(y_test.values, y_pred, 'ko')
plt.plot([0,600000],[0,600000])

## Using multiple features for regression:  Multiple Regression 

In [ ]:
house_data.shape

In [ ]:
target = house_data['SalePrice']
features = house_data.drop('SalePrice', axis=1)

In [ ]:
features.info()

It may be good to use as much data as possible.  On the other hand, maybe we want to keep our model more understandable by paring down the number of features, maybe we don't want to use the time and/or computational resources necessary to train on ALL the data, maybe some features have lots of nulls / errors / outliers that make them of more questionable merit...

I'm only going to keep numerical columns for now.

In [ ]:
features = features.select_dtypes(include='number').copy()

And I'm going to drop the columns that don't have 1460 non-null values.

In [ ]:
features = features.drop(['GarageYrBlt','MasVnrArea','LotFrontage'],axis=1)

In [ ]:
features.columns

### Yellowbrick

* "Yellowbrick extends the Scikit-Learn API to make model selection and hyperparameter tuning easier. Under the hood, it’s using Matplotlib."
* https://www.scikit-yb.org/en/latest/

In [ ]:
visualizer = FeatureCorrelation(labels = list(features.columns), sort=True)
visualizer.fit(features, target)
visualizer.show()

### Select K-Best features to predict price of houses

Scikit-Learn has methods for selecting the "best" features.  For example, the following will use `f_regression` to do "univariate linear regression tests returning F-statistic and p-values" in order to select a set of best features.
* [feature_selection with f_regression](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)

In [ ]:
best6 = SelectKBest(f_regression, k=6).fit(features, target)

In [ ]:
best6

In [ ]:
help(best6)

In [ ]:
best6.get_feature_names_out()

In [ ]:
best6.get_support()

In [ ]:
bestcolumns = features.columns[best6.get_support()]
bestcolumns

In [ ]:
myfeatures = features[bestcolumns]
myfeatures.head()

In [ ]:
myfeatures.describe()

### Another aside on being careful with feature variables: multicollinearity

When we consider feature engineering, we'll want to consider whether some of the variables are dependent.  And exactly what happens when we have multicollinearity of the variables.

Multicollinearity may not have an effect on the predictive power of our model, but it can affect the variance of our coefficient estimates, lead to larger confidence intervals, and make it more difficult to interpret the predictors of our final model.

Here we briefly show how to identify whether variables might suffer from collinearity by measuring the variance inflation factor (VIF) -- values of 1 are ignorable, < 5 are reasonable, and > 5 should be dealt with.

In [ ]:
myfeatures.values

In [ ]:
vif = pd.DataFrame()
vif['VIF'] = [round(variance_inflation_factor(myfeatures.values, i), 1) for i in range(myfeatures.shape[1])]
vif['variable'] = myfeatures.columns

In [ ]:
vif.sort_values(by='VIF', ascending=False)

## Back to regression:

In [ ]:
X = pd.DataFrame(data=myfeatures, columns=myfeatures.columns)
y = target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
linear_regression = LinearRegression()

In [ ]:
linear_regression.fit(X_train, y_train)

In [ ]:
y_pred = linear_regression.predict(X_test)

In [ ]:
df = pd.DataFrame({'test': y_test, 'Predicted': y_pred})

In [ ]:
df.head()

In [ ]:
score = linear_regression.score(X_test, y_test)
r2score = r2_score(y_test, y_pred)
stddevscore = np.sqrt(mean_squared_error(y_test, y_pred))

print('Score: {}'.format(score))
print('r2_score: {}'.format(r2score))
print('stddev_score: {}'.format(stddevscore))

In [ ]:
linear_regression.coef_

In [ ]:
linear_regression.intercept_

### Scaling

Scaling of feature values is another feature engineering concept that is useful.

Feature scaling can be necessary for applying certain algorithms, it may not matter at all in other cases, sometimes it can make algorithms run faster, it can give a better error surface shape, it can prevent optimization algorithms from getting stuck in local minima, it can reduce the collinearity of two variables, ....

Here we have features that have very different scales, as can be seen by the coefficients above that have very different scales.

In [ ]:
X = pd.DataFrame(data=myfeatures, columns=myfeatures.columns)
y = target

In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Wait!  Danger:**  we've scaled our data before doing the test/train split.  Information about the training set may therefore have leaked into the test set.

In [ ]:
print(X_train.mean())
print(X_train.std())
print(X_test.mean())
print(X_test.std())

In [ ]:
print(np.concatenate((X_train,X_test)).mean())
print(np.concatenate((X_train,X_test)).std())

This means the whole dataset has been standard scaled, but we only want to standard scale our training data.

REDO:

In [ ]:
X = pd.DataFrame(data=myfeatures, columns=myfeatures.columns)
y = target

In [ ]:
X.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
print(X_train.mean())
print(X_train.std())
print(X_test.mean())
print(X_test.std())

In [ ]:
print(np.concatenate((X_train,X_test)).mean())
print(np.concatenate((X_train,X_test)).std())

Ok, better.

In [ ]:
linear_regression = LinearRegression()

In [ ]:
linear_regression.fit(X_train, y_train)

In [ ]:
y_pred = linear_regression.predict(X_test)

In [ ]:
df = pd.DataFrame({'test': y_test, 'Predicted': y_pred})

In [ ]:
df.head()

In [ ]:
score = linear_regression.score(X_test, y_test)
r2score = r2_score(y_test, y_pred)
stddevscore = np.sqrt(mean_squared_error(y_test, y_pred))

print('Score: {}'.format(score))
print('r2_score: {}'.format(r2score))
print('stddev_score: {}'.format(stddevscore))

The performance here is the same, but now the coefficients we find are on the same order:

In [ ]:
linear_regression.coef_

In [ ]:
linear_regression.intercept_

## Other algorithms and libraries

In [ ]:
X_train

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=500, 
                                 max_leaf_nodes=8, 
                                 n_jobs=-1,
                                 random_state=42)
rf_reg.fit(X_train, y_train)

test_score = rf_reg.score(X_test, y_test)
print(f"R2 of Random Forest: {test_score:.2f}")

preds = rf_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
cv_grid = GridSearchCV(RandomForestRegressor(n_jobs=-1,random_state=42),
                       param_grid = {
                           'max_depth' : [5,10,20],
                           'n_estimators' : [200,500],
                           'max_leaf_nodes' : [8, 16, 32]
                       })
cv_grid.fit(X_train, y_train)
cv_grid.best_params_

In [ ]:
y_predict = cv_grid.predict(X_test)
r2score = r2_score(y_test,y_predict)
print('R2 of the best Random Forest regressor after CV is %.2f' % (r2score))

In [ ]:
plt.barh(X_train.columns, rf_reg.feature_importances_)

### XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xg_reg = xgb.XGBRegressor()

In [ ]:
params = {"objective":["reg:squarederror"],
                  'colsample_bytree': [0.3, 1.0],
                  'learning_rate': [0.1,0.3,0.5],
                  'max_depth': [5,10,20],
                  'max_leaves': [8,16,32],
                  'alpha': [5,10]}

xg_reg_opt = GridSearchCV(xg_reg, params, n_jobs=-1)

xg_reg_opt.fit(X_train, y_train)

test_score = xg_reg_opt.score(X_test, y_test)
print(f"R2 of Linear Regression: {test_score:.2f}")

In [ ]:
xg_reg_opt.best_params_

### Tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ])

model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])

model.fit(X_train, y_train,
          epochs=1000, batch_size=16, verbose=0)
          # epochs=130, batch_size=16)

test_mse_score, test_mae_score = model.evaluate(X_test, y_test)

In [ ]:
np.sqrt(test_mse_score)

In [ ]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1)
    ])
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model
    
k = 4
num_val_samples = len(X_train) // k
num_epochs = 1000
all_train_histories = []
all_val_histories = []

for i in range(k):

    print(f"Processing fold #{i}")
    
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)
    
    model = build_model()
    
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=150, verbose=0)
    
    train_history = history.history["mae"]
    all_train_histories.append(train_history)
    val_history = history.history["val_mae"]
    all_val_histories.append(val_history)

In [ ]:
average_train_history = [
    np.mean([x[i] for x in all_train_histories]) for i in range(num_epochs)]
average_val_history = [
    np.mean([x[i] for x in all_val_histories]) for i in range(num_epochs)]

plt.plot(range(50, len(average_train_history)), average_train_history[50:], 'r')
plt.plot(range(50, len(average_val_history)), average_val_history[50:], 'g')
plt.xlabel("Epochs")
plt.ylabel("Train/Validation MAE")
plt.show()

In [ ]:
y_pred = model.predict(X_test)
print('R2 score: ',r2_score(y_test, y_pred))